In [1]:
import os
import json
import pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [2]:
np.random.seed(21)

In [3]:
from keras import backend as K
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.layers import Dense, AvgPool2D, Conv2D, Reshape
from keras.models import Model, load_model
from keras.optimizers import RMSprop, SGD
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import Iterator
from keras.utils.np_utils import to_categorical
import tensorflow as tf
sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [4]:
class NonValidPatch(Exception):
    pass

In [16]:
def get_block_loc(shape, x, y, target_size=(224, 224), n_blocks=(4,4), overlap=(448,448)):
    h, w = shape
    w_block = (w + (n_blocks[1] - 1) * overlap[1]) // n_blocks[1]
    h_block = (h + (n_blocks[0] - 1) * overlap[0]) // n_blocks[0]
    for by in range(n_blocks[0]):
        y_start = by * (h_block - overlap[0])
        y_end = y_start + h_block + 1
        for bx in range(n_blocks[1]):
            x_start = bx * (w_block - overlap[1])
            x_end = x_start + w_block + 1
            
            if x_start <= x < x_end and y_start <= y < y_end and\
            x_start <= x + target_size[1] - 1 < x_end and y_start <= y + target_size[0] - 1 < y_end:
                return bx + by * n_blocks[0], x - x_start, y - y_start
    raise NonValidPatch("Can't find block...??")

In [17]:
class StreamStats(object):
    """ See https://www.johndcook.com/blog/standard_deviation/
    """
    def __init__(self):
        self.M = None
        self.S = None
        self.k = 0
        self.min = None
        self.max = None
        
    def update(self, x):
        self.k += 1
        if self.k == 1:
            self.M = x
            self.S = 0
            self.min = x
            self.max = x
        else:
            prevM = self.M
            prevS = self.S
            self.M = prevM + (x - prevM)/self.k
            self.S = prevS + (x - prevM) * (x - self.M)
            self.min = np.minimum(x, self.min)
            self.max = np.maximum(x, self.max)
            
    def mean(self):
        return self.M
        
    def variance(self):
        if self.k - 1 > 0:
            return self.S / (self.k - 1)
        else:
            return 0
    
    def std(self):
        return np.sqrt(self.variance())
    
    def minimum(self):
        return self.min
    
    def maximum(self):
        return self.max

In [54]:
class NoPupsIterator(Iterator):
    """Iterator yielding training samples of subadult_males
    :param root_dir: Directory containing training images, and dots.
    :param image_ids: Set of image ids to use to sample patches.
    :param class_weights: Weights for each class.
    :param n_samples_per_image: Number of patches to sample on each image.
    :param target_size: Size of the patches sampled.
    :param batch_size: Number of patches sampled per batch
    :param shuffle: Boolean, whether to shuffle the data between epochs.
    :param seed: Random seed for data shuffling.
    :return batch_x, batch_x. 
        batch_x is a (batch_size, target_size[0], target_size[1], 3) array
        batch_x is a (batch_size, target_size[0], target_size[1], 1) array if output_counts is False
        otherwise, it is a (batch_size, 5) array.
    """

    def __init__(self, root_dir, image_ids,
                 class_weights = None,
                 n_samples_per_image=160,
                 target_size=(92, 92),
                 batch_size=64, shuffle=True, seed=42, debug_dir=None):
        self.n_samples_per_block = 16
        self.n_sealion_types = 5
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.debug_dir = debug_dir
        # Normalize to use class_weights as a probability distribution.
        if class_weights:
            self.class_weights = np.asarray(class_weights)/np.sum(class_weights)
        else:
            self.class_weights = np.ones((self.n_sealion_types+1))/(self.n_sealion_types + 1)
            
        self.n_samples_per_image = n_samples_per_image
        self.target_size = target_size
        self.n_indices = len(self.image_ids) * self.n_samples_per_image
                 
        super(NoPupsIterator, self).__init__(self.n_indices, batch_size//self.n_samples_per_block, shuffle, seed)
        
    def get_class_distribution(self, n_batches=100):
        counts = defaultdict(int)
        for _ in range(n_batches):
            _, by = self.next()
            by = np.argmax(by, axis=-1)
            cls, cnts = np.unique(by.ravel(), return_counts=True)
            for c, cnt in zip(cls, cnts):
                counts[c] += cnt
        return counts
        
    def normalize_input(self, x_bgr):
        x = x_bgr.copy()
        x[..., 0] -= 103.939
        x[..., 1] -= 116.779
        x[..., 2] -= 123.68
        return x
    
    def denormalize_input(self, x_normed):
        x = x_normed.copy()
        x[..., 0] += 103.939
        x[..., 1] += 116.779
        x[..., 2] += 123.68
        return x

    def random_transform(self, im):
        flip_hor = np.random.randint(0, 2)
        flip_ver = np.random.randint(0, 2)
        if flip_hor == 1:
            im = cv2.flip(im, 0)
        if flip_ver == 1:
            im = cv2.flip(im, 1)
        return im
    
    def get_weights(self, dots):
        # Set probability to 0 if some sealion type is not in the block
        current_weigths = self.class_weights.copy()
        for i in range(self.n_sealion_types):
            if not dots[i]:
                current_weigths[i] = 0
        current_weigths /= np.sum(current_weigths)
        return current_weigths
    
    def sample_position(self, shape, dots, current_weigths, size):
        # Choose an output class randomly
        output_class = np.random.choice(self.n_sealion_types + 1, size=(1, ), p=current_weigths)[0]
        # Sample a location, either for background or for a sealion.
        if output_class == self.n_sealion_types:
            # avoid bg with pups in it
            return self.sample_bg(shape, dots, size), output_class
        else:
            return self.sample_dot(shape, dots[output_class], size), output_class

    def get_dots_in_block(self, bid, shape, dots, n_blocks=(4,4), overlap=(448,448)):
        h, w = shape
        w_block = (w + (n_blocks[1] - 1) * overlap[1]) // n_blocks[1]
        h_block = (h + (n_blocks[0] - 1) * overlap[0]) // n_blocks[0]
        
        bx = bid % n_blocks[0]
        by = bid // n_blocks[0]
        
        y_start = by * (h_block - overlap[0])
        y_end = y_start + h_block + 1
        x_start = bx * (w_block - overlap[1])
        x_end = x_start + w_block + 1
        
        dots_in_block = [[] for _ in range(self.n_sealion_types)]
        for i, ds in enumerate(dots):
            for (x, y) in ds:
                if x_start <= x < x_end and y_start <= y < y_end:
                    dots_in_block[i].append((x - x_start, y - y_start))
        return dots_in_block
        
    def get_random_block(self, image_id, shape, dots, current_weigths):
        while True:
            try:
                (x, y), output_class = self.sample_position(shape, dots, current_weigths, self.target_size)
                
                # Get the corresponding image block, and (x, y) in this block
                bid, x, y = get_block_loc(shape, x, y)
                
                # Load the block and check if it is valid
                uid = "{iid}_{bid}".format(iid=image_id, bid=bid)
                img = cv2.imread(os.path.join(self.root_dir, "TrainBlock", uid + ".jpg"))
                if img is not None:
                    return bid, img
            except NonValidPatch:
                continue
        
    
    def sample(self, shape, dots, image_id):
        # if more than 30% of the patch is masked, reject it
        threshold_masked = 0.3 
        
        # If we are stuck on a "bad" block, we retry from scratch
        while True:
            # Set probability to 0 if some sealion type is not in the block
            current_weigths = self.get_weights(dots)
        
            # Get a block randomly (but using the dots and the sampling weights)
            bid, img = self.get_random_block(image_id, shape, dots, current_weigths)
                
            # Recompute dots in the blocks, and sampling weights
            dots_block = self.get_dots_in_block(bid, shape, dots)
            current_weigths = self.get_weights(dots_block)

            # Now, sample n_samples_per_block patches from it
            n_samples = 0
            bx = np.zeros((self.n_samples_per_block, self.target_size[0], self.target_size[1], 3))
            by = np.zeros((self.n_samples_per_block, ))

            # Stop if we try too many times, 
            max_iterations = self.n_samples_per_block * 5
            current_iteration = 0
            while n_samples < self.n_samples_per_block and current_iteration < max_iterations:
                current_iteration += 1
                try:
                    zoom_max = 1.2
                    zoom_min = 0.8
                    hmin = int(self.target_size[0]/zoom_max)
                    hmax = int(self.target_size[0]/zoom_min)
                    wmin = int(self.target_size[1]/zoom_max)
                    wmax = int(self.target_size[1]/zoom_min)
                    
                    h = np.random.randint(hmin, hmax, size=(1,))[0]
                    w = np.random.randint(wmin, wmax, size=(1,))[0]
 
                    (x, y), output_class = self.sample_position(img.shape[:2], dots_block, current_weigths, (h, w))
                    img_patch = img[y:y+h, x:x+w,:]

                    img_patch = cv2.resize(img_patch, dsize=(self.target_size[1], self.target_size[0]))
                    masked_pixels = np.count_nonzero(img_patch == 0)
                    total_pixels = img_patch.shape[0] * img_patch.shape[1]
                    if img_patch.shape[0] != self.target_size[0] or img_patch.shape[1] != self.target_size[1]:
                        continue
                    if masked_pixels/total_pixels < threshold_masked:
                        bx[n_samples, ...] = self.random_transform(img_patch)
                        by[n_samples] = output_class if output_class < 4 else 4
                        n_samples += 1
                except NonValidPatch:
                    continue
                    
            if current_iteration < max_iterations:
                return bx, by
            # else, we tried too many times, let's get another block.
        
        
    def contains_dots(self, xstart, ystart, dots, margin):
        x1 = xstart - margin
        y1 = ystart - margin
        x2 = xstart + self.target_size[1] + margin
        y2 = ystart + self.target_size[0] + margin
        for ds in dots:
            for (x, y) in ds:
                if x1 <= x < x2 and y1 <= y < y2:
                    return True
        return False
    
    def sample_bg(self, shape, dots, size):
        margin = 40 # more than half of the sealion expected size
        max_iterations = 10
        current_iteration = 0
        while current_iteration < max_iterations:
            if shape[1] - size[1] <= 0 or shape[0] - size[0] <= 0:
                raise NonValidPatch("Cant' find background")
            x = np.random.randint(0, shape[1] - size[1], size=(1,))[0]
            y = np.random.randint(0, shape[0] - size[0], size=(1,))[0]
            if not self.contains_dots(x, y, dots, margin):
                return x, y
            current_iteration += 1
        raise NonValidPatch("Cant' find background")
    
    def sample_dot(self, shape, dots, size):
        margin = size[0]//10
        
        rand_index = np.random.choice(len(dots), size=(1,))[0]
        rand_dot = dots[rand_index]
        
        min_x = max(0, rand_dot[0] - size[1]//2 - margin)
        max_x = max(0, min(shape[1] - size[1], rand_dot[0] - size[1]//2 + margin))
        
        min_y = max(0, rand_dot[1] - size[0]//2 - margin)
        max_y = max(0, min(shape[0] - size[0], rand_dot[1] - size[0]//2 + margin))
        
        if min_x > max_x:
            max_x, min_x = min_x, max_x
        if min_y > max_y:
            max_y, min_y = min_y, max_y 
            
        if min_x == max_x or min_y == max_y:
            raise NonValidPatch()
           
        x = np.random.randint(min_x, max_x, size=(1,))[0]
        y = np.random.randint(min_y, max_y, size=(1,))[0]
        
        return x, y
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                
        batch_x = np.zeros((current_batch_size * self.n_samples_per_block, self.target_size[0], self.target_size[1], 3), dtype=K.floatx())
        batch_y = np.zeros((current_batch_size * self.n_samples_per_block), dtype=np.int32)
        
        # For each index, we load the data and sample randomly n_successive_samples patches
        for i, j in enumerate(index_array):
            index = j // self.n_samples_per_image
            image_id = self.image_ids[index]
            with open(os.path.join(self.root_dir, "TrainDots", str(image_id) + ".pkl"), "rb") as pfile:
                dots = pickle.load(pfile)
            with open(os.path.join(self.root_dir, "TrainShape", str(image_id) + ".pkl"), "rb") as pfile:
                shape = pickle.load(pfile)
                
            x, y = self.sample(shape, dots, image_id)
            batch_x[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block, ...] = x
            batch_y[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block] = y 

        if self.debug_dir:
            for i in range(current_batch_size):
                cv2.imwrite(os.path.join(self.debug_dir, "patch_{}.jpg".format(i)), batch_x[i])
                
        permut = np.random.permutation(batch_x.shape[0])
        return self.normalize_input(batch_x[permut, ...]), to_categorical(batch_y[permut,...], num_classes=5).reshape((-1, 1, 1, 5))

In [55]:
with open("../data/sealion/train.json", "r") as jfile:
    train_ids = json.load(jfile)
train_ids = [int(s) for s in train_ids]

with open("../data/sealion/val.json", "r") as jfile:
    val_ids = json.load(jfile)
val_ids = [int(s) for s in val_ids]


In [56]:
sealion_types = ["adult_males", 
    "subadult_males",
    "adult_females",
    "juveniles",
    "pups"]

In [72]:
class_weights = [0.1,  0.1, 0.1, 0.1, 0, 0.1]

In [73]:
trainPatchesGenerator = NoPupsIterator("/home/lowik/sealion/data/sealion/", train_ids, class_weights=class_weights)

In [74]:
valPatchesGenerator = NoPupsIterator("/home/lowik/sealion/data/sealion/", val_ids, class_weights=class_weights)

Create model

In [88]:
from keras.applications.resnet50 import ResNet50

In [89]:
base_model = ResNet50(weights='imagenet', include_top=False, pooling=None)

In [90]:
h = base_model.layers[-2]
sealion_prediction = Conv2D(5, (3, 3), activation="softmax")(h.output)

In [91]:
sealion_net = Model(inputs=[base_model.input], outputs=[sealion_prediction])

In [92]:
sealion_net.predict(np.ones((1, 92, 92, 3))).shape

(1, 1, 1, 5)

In [93]:
sealion_net.predict(np.ones((1, 92*2, 92*2, 3))).shape

(1, 4, 4, 5)

In [94]:
for layer in base_model.layers:
    layer.trainable = False

In [95]:
for bx, by in trainPatchesGenerator:
    break

In [96]:
sealion_net.predict(bx)

array([[[[  1.19998351e-01,   8.60839784e-01,   2.77264463e-03,
            1.55479144e-02,   8.41396919e-04]]],


       [[[  7.79132098e-02,   4.52489227e-01,   1.17500365e-01,
            2.19280049e-01,   1.32817060e-01]]],


       [[[  1.51233450e-02,   6.69259608e-01,   2.67610103e-01,
            4.29757163e-02,   5.03114797e-03]]],


       [[[  4.85859424e-01,   2.68659145e-01,   9.93461162e-02,
            1.00647219e-01,   4.54880595e-02]]],


       [[[  1.70679137e-01,   2.77053535e-01,   2.72047877e-01,
            2.51753926e-01,   2.84654796e-02]]],


       [[[  6.21781219e-03,   8.96522343e-01,   4.22375575e-02,
            4.90865335e-02,   5.93572343e-03]]],


       [[[  1.04864925e-01,   1.32874027e-03,   8.93055499e-01,
            3.94794879e-05,   7.11383123e-04]]],


       [[[  1.88481230e-02,   1.39194906e-01,   9.57535859e-03,
            8.23085368e-01,   9.29619558e-03]]],


       [[[  3.68588090e-01,   1.01822913e-01,   2.36121744e-01,
            1.46

In [97]:
from collections import defaultdict

In [85]:
trainPatchesGenerator.get_class_distribution(100)

defaultdict(int, {0: 1241, 1: 943, 2: 1039, 3: 1025, 4: 2152})

In [98]:
sgd = SGD(lr=0.002, momentum=0.9, decay=1e-6, nesterov=True)
sealion_net.compile(optimizer=sgd, loss=categorical_crossentropy, metrics=['accuracy'])

In each epoch, we will do 32 * 300 = 9600 samples

In [99]:
h = sealion_net.fit_generator(trainPatchesGenerator, 300, epochs=3, verbose=1, callbacks=None, validation_data=valPatchesGenerator, validation_steps=50, class_weight=None, max_q_size=10, workers=1, pickle_safe=False, initial_epoch=0)

Epoch 1/3
300/300 [==============================] - 1066s - loss: 3.2209 - acc: 0.5965 - val_loss: 3.5236 - val_acc: 0.6503
Epoch 2/3
 45/300 [===>..........................] - ETA: 711s - loss: 3.7646 - acc: 0.5806

KeyboardInterrupt: 

In [ ]:
model.save("../data/sealion/xception_fc_3_epochs_300steps_last_layer.h5")

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
sgd = SGD(lr=0.0001, momentum=0.9, decay=0.0005, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
h = model.fit_generator(trainPatchesGenerator, 300, epochs=3, verbose=1, callbacks=None, validation_data=valPatchesGenerator, validation_steps=50, class_weight=None, max_q_size=10, workers=1, pickle_safe=False, initial_epoch=0)

In [ ]:
model.save("../data/sealion/xception_fc_3_3_epochs_300steps.h5")

In [ ]:
for bx, by in valPatchesGenerator:
    break

In [ ]:
by_pred = model.predict(bx)

In [ ]:
t = np.ones((1, 91*2, 91*2, 3))

In [ ]:
tp = model.predict(t)

In [ ]:
tp.shape

In [ ]:
tp

In [ ]:
im = cv2.imread("../data/test_fcb.png")

In [ ]:
91 * 6

In [ ]:
im.shape

In [ ]:
im_normed = trainPatchesGenerator.normalize_input(im.astype(np.float32))

In [ ]:
im_normed = im_normed.reshape((1, 546, 546, 3))

In [ ]:
py = model.predict(im_normed)

In [ ]:
np.argmax(py, axis=-1).shape

In [ ]:
plt.imshow(im)

In [ ]:
i = 8
plt.imshow(trainPatchesGenerator.denormalize_input(bx[i, ...]))
print("Predict probas: ", by_pred[i, ...])

In [ ]:
im = cv2.imread("../data/sealion/TrainBlock/872_2.jpg")

In [ ]:
plt.imshow(im)

In [ ]:
im = im.reshape((1, ) + im.shape)

In [ ]:
im.dtype

In [ ]:
im_norm = trainPatchesGenerator.normalize_input(im.astype(np.float32))

In [ ]:
y_pred = model.predict(im_norm)

In [ ]:
im_norm.shape

In [ ]:
1741/91

In [ ]:
y_pred.shape

In [ ]:
y_class = np.argmax(y_pred, axis=-1)

In [ ]:
y_class.shape

In [ ]:
y_pred

In [ ]:
y_class

In [ ]:
plt.imshow(y_class[0,:,:])